In [1]:
import ipyparallel as ipp
c = ipp.Client()

In [2]:
c.ids

[0, 1, 2, 3]

In [3]:
c


In [4]:
!ls ~/.ipython/profile_default/security

ipcontroller-client.json  ipcontroller-engine.json


In [ ]:
# %load ~/.ipython/profile_default/security/ipcontroller-engine.json
{
  "control": 51647, 
  "task": 47914, 
  "hb_ping": 36328, 
  "mux": 59650, 
  "pack": "json", 
  "hb_pong": 56864, 
  "ssh": "", 
  "key": "3d1d9412-87ce-4134-97fc-84b5bd94e911", 
  "registration": 47569, 
  "interface": "tcp://127.0.0.1", 
  "iopub": 49887, 
  "signature_scheme": "hmac-sha256", 
  "unpack": "json", 
  "location": "192.168.0.6"
}

In [6]:
c[:]

<DirectView [0, 1, 2, 3]>

In [7]:
"Hola alumnos"

'Hola alumnos'

In [8]:
c[0].apply_sync(lambda : "Hola alumnos") # utiliza solo un motor

'Hola alumnos'

In [9]:
c[:].apply_sync(lambda : "Hola muchos alumnos") # utiliza todos los motores

['Hola muchos alumnos',
 'Hola muchos alumnos',
 'Hola muchos alumnos',
 'Hola muchos alumnos']

In [10]:


dview = c[:] # asigna todos los motores a la variable dview
dview.apply_sync(lambda : "Hola muchos alumnos")



['Hola muchos alumnos',
 'Hola muchos alumnos',
 'Hola muchos alumnos',
 'Hola muchos alumnos']

In [11]:
dview


<DirectView [0, 1, 2, 3]>

In [12]:
dview = c.direct_view() # es equivalente a dview = c[:]

In [13]:
import os
dview.apply_sync(os.getpid)

[6634, 6630, 6632, 6637]

In [14]:
import socket
dview.apply_sync(socket.gethostname)

['debian', 'debian', 'debian', 'debian']

In [15]:
dview.map_sync(lambda x, y, z: x + y + z, range(10), range(10), range(10))

[0, 3, 6, 9, 12, 15, 18, 21, 24, 27]

In [16]:
c[::2].map_sync(lambda x, y, z: x + y + z, range(10), range(10), range(10))

[0, 3, 6, 9, 12, 15, 18, 21, 24, 27]

In [17]:
c[2].map_sync(lambda x, y, z: x + y + z, range(10), range(10), range(10))

[0, 3, 6, 9, 12, 15, 18, 21, 24, 27]

In [18]:
import numpy as np

In [20]:
lview = c.load_balanced_view()

In [21]:
lview.map_sync(lambda x: sum(x), np.random.random((10,1000000)))

[499973.21881848958,
 499980.00083828752,
 499551.17637335503,
 499651.71707058494,
 499924.64285466797,
 500411.36398312345,
 499845.92843455228,
 499833.47309662774,
 499833.46512978547,
 499631.18174379104]

In [22]:


def cube(x):
    print x**3
    return x**3



cube(2)



8


8

In [23]:
lista1=[cube(x) for x in range(1,7)]
lista1

1
8
27
64
125
216


[1, 8, 27, 64, 125, 216]

In [24]:
map(cube, range(1,7))

1
8
27
64
125
216


[1, 8, 27, 64, 125, 216]

In [25]:
serial = map(cube, range(1,70))

1
8
27
64
125
216
343
512
729
1000
1331
1728
2197
2744
3375
4096
4913
5832
6859
8000
9261
10648
12167
13824
15625
17576
19683
21952
24389
27000
29791
32768
35937
39304
42875
46656
50653
54872
59319
64000
68921
74088
79507
85184
91125
97336
103823
110592
117649
125000
132651
140608
148877
157464
166375
175616
185193
195112
205379
216000
226981
238328
250047
262144
274625
287496
300763
314432
328509


In [26]:
parallel = dview.map_sync(cube, range(1,70))

In [27]:
serial == parallel

True

In [28]:
%timeit map(cube, range(1,70))
%timeit dview.map_sync(cube, range(1,70))

1
8
27
64
125
216
343
512
729
1000
1331
1728
2197
2744
3375
4096
4913
5832
6859
8000
9261
10648
12167
13824
15625
17576
19683
21952
24389
27000
29791
32768
35937
39304
42875
46656
50653
54872
59319
64000
68921
74088
79507
85184
91125
97336
103823
110592
117649
125000
132651
140608
148877
157464
166375
175616
185193
195112
205379
216000
226981
238328
250047
262144
274625
287496
300763
314432
328509
1
8
27
64
125
216
343
512
729
1000
1331
1728
2197
2744
3375
4096
4913
5832
6859
8000
9261
10648
12167
13824
15625
17576
19683
21952
24389
27000
29791
32768
35937
39304
42875
46656
50653
54872
59319
64000
68921
74088
79507
85184
91125
97336
103823
110592
117649
125000
132651
140608
148877
157464
166375
175616
185193
195112
205379
216000
226981
238328
250047
262144
274625
287496
300763
314432
328509
1
8
27
64
125
216
343
512
729
1000
1331
1728
2197
2744
3375
4096
4913
5832
6859
8000
9261
10648
12167
13824
15625
17576
19683
21952
24389
27000
29791
32768
35937
39304
42875
46656
50653
54872
59319


In [30]:
dview.map_sync(cube, range(1,7))

[1, 8, 27, 64, 125, 216]

In [31]:
def multi(a,b):
    return a*b
dview.map_sync(multi, [0,1,3,5],[2,4,6,8])

[0, 4, 18, 40]

In [32]:
PrimeQ=lambda x: all(x % i != 0 for i in range(int(x**0.5)+1)[2:]) and x>1

In [33]:
PrimeQ(2)

True

In [34]:
PrimeQ(137)

True

In [35]:
PrimeQ(135)

False

In [36]:
def potenciar(numero):
    return 2**numero-1
    

In [48]:
def repetir(numero , limite):
    if numero < limite:
        potencia = potenciar(numero)
        if(PrimeQ(potencia)):
            print  potencia
        numero =  numero +1
     
    repetir(numero, limite)
    
    
    
    
    
    

<h1>Ejercicios</h1>

Determinar si un numero de la forma $2^n -1$, donde $n$ es un entero, es primo o no. Los primos que tienen esta forma se llaman primos de Mersenne.

*Escribir una funcion que determine si el numero dado por $2^n -1$ es primo (usar la funcion anterior)


In [44]:
repetir(1,51)


3
7
31
127
8191
131071
524287
2147483647


In [46]:
%time repetir(1,51)


3
7
31
127
8191
131071
524287
2147483647
CPU times: user 3.84 s, sys: 616 ms, total: 4.45 s
Wall time: 4.45 s


In [1]:
%time repetir(1,4)

NameError: name 'repetir' is not defined